In [3]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import requests
import json
import numpy as np
import joblib
import tensorflow as tf
from tensorflow.keras.models import load_model




# Function to check if the market is open
def is_market_open():
    ticker = yf.Ticker("AAPL")
    market_state = ticker.info.get("marketState", "CLOSED").upper()
    return market_state == "REGULAR"

# Function to fetch stock data for the last 10 trading days with precomputed SMA & EMA
def get_last_10_days_stock_data(ticker_symbol):
    today = datetime.now().date()
    end_date = today
    start_date = today - timedelta(days=30)# Fetch 30 days to compute SMA & EMA properly

    stock_data = yf.download(ticker_symbol, start=start_date, end=end_date)

    # Compute SMA and EMA before filtering
    stock_data["SMA_7"] = stock_data["Close"].rolling(window=7).mean()
    stock_data["EMA_7"] = stock_data["Close"].ewm(span=7, adjust=False).mean()
    

    # Now filter only the last 10 trading days
    last_10_days = stock_data.tail(10).copy()

    # If the market is open, update the latest close price with live trading price
    if is_market_open():
        ticker = yf.Ticker(ticker_symbol)
        live_price = ticker.history(period="1d")["Close"].iloc[-1]  # Get the latest live price
        last_10_days.iloc[-1, last_10_days.columns.get_loc("Close")] = live_price  # Update closing price

    return last_10_days

# Fetch and display the last 10 days of stock data for AAPL
ticker_symbol = "AAPL"
last_10_days_data = get_last_10_days_stock_data(ticker_symbol)
last_10_days_data.columns = [col[0] if col[1] == 'AAPL' else col[0] for col in last_10_days_data.columns]
# Reset index to bring 'Date' as a normal column
last_10_days_data = last_10_days_data.reset_index()

print(last_10_days_data)



# Alpha Vantage API Key (Replace with your own)
API_KEY = "4ENQ6BPKCZDJLLJ7"

# Function to fetch the latest 7 days of news for AAPL
def get_recent_news(ticker="AAPL"):
    url = f"https://www.alphavantage.co/query"
    params = {
        "function": "NEWS_SENTIMENT",
        "tickers": ticker,
        "apikey": API_KEY
    }

    response = requests.get(url, params=params)
    data = response.json()

    if "feed" not in data:
        print("Error: No news data found.")
        return None

    # Get today's date and calculate the last 7 days
    today = datetime.utcnow()
    seven_days_ago = today - timedelta(days=7)

    # Extract relevant news articles from the last 7 days
    news_data = []
    for article in data["feed"]:
        time_published = datetime.strptime(article["time_published"], "%Y%m%dT%H%M%S")
        
        if time_published >= seven_days_ago:
            news_data.append({
                "time_published": time_published,
                "title": article["title"],
                "summary": article["summary"],
            })

    # Convert to DataFrame for better readability
    news_df = pd.DataFrame(news_data)
    
    return news_df

# Fetch recent 7 days of news for AAPL
news_df = get_recent_news("AAPL")
print(news_df)




# API Credentials
API_TOKEN = "67d7c6911d7fe5.77162755"
ticker = "aapl.us"
end_date = datetime.today().strftime('%Y-%m-%d')
start_date = (datetime.today() - timedelta(days=30)).strftime('%Y-%m-%d')

# API URL for sentiment data
url = f"https://eodhd.com/api/sentiments?s={ticker}&from={start_date}&to={end_date}&api_token={API_TOKEN}&fmt=json"

# Fetch API response
response = requests.get(url)
data = response.json()

# Convert ticker to uppercase for key matching
ticker_upper = ticker.upper()

# Dictionary to store categorized sentiment data
sentiment_data = {}

# Function to estimate sentiment probabilities based on normalized sentiment score
def estimate_sentiment_distribution(score):
    if score > 0.5:
        return {"positive": round(score, 2), "neutral": round(1 - score, 2), "negative": 0.0}
    elif -0.5 <= score <= 0.5:
        return {"positive": 0.0, "neutral": 1.0, "negative": 0.0}
    else:
        return {"positive": 0.0, "neutral": round(1 + score, 2), "negative": round(abs(score), 2)}

# Check if sentiment data exists
if ticker_upper in data and data[ticker_upper]:
    for entry in data[ticker_upper]:
        date = entry["date"]
        score = entry["normalized"]  # FIXED KEY ERROR
        sentiment_data[date] = estimate_sentiment_distribution(score)

    print("✅ Sentiment data categorized successfully!")
else:
    print("❌ No sentiment data available for the given date range.")

# Print categorized sentiment data
print(json.dumps(sentiment_data, indent=4))


# Extract the dates from last_10_days_data
last_10_days_dates = set(last_10_days_data["Date"].astype(str))

# Filter sentiment_data to keep only matching dates
filtered_sentiment_data = {date: sentiment_data[date] for date in sentiment_data if date in last_10_days_dates}

# Print the filtered sentiment data
print(json.dumps(filtered_sentiment_data, indent=4))



# Convert filtered_sentiment_data dictionary to DataFrame
sentiment_df = pd.DataFrame.from_dict(filtered_sentiment_data, orient="index").reset_index()
sentiment_df.columns = ["Date", "positive_sentiment", "neutral_sentiment", "negative_sentiment"]

# Convert Date column to datetime format
sentiment_df["Date"] = pd.to_datetime(sentiment_df["Date"])
last_10_days_data["Date"] = pd.to_datetime(last_10_days_data["Date"])

# Merge stock data with sentiment data
merged_df = pd.merge(last_10_days_data, sentiment_df, on="Date", how="left")

# Display the final DataFrame
print(merged_df)



# Rename columns to match the model's expected input format
merged_df = merged_df.rename(columns={
    'Close': 'close',
    'High': 'high',
    'Low': 'low',
    'Open': 'open',
    'Volume': 'volume',
    'SMA_7': 'SMA',
    'EMA_7': 'EMA',
    'positive_sentiment': 'positive',
    'neutral_sentiment': 'neutral',
    'negative_sentiment': 'negative'
})

# Add missing columns with the correct default values
merged_df['GDP'] = 29719.647
merged_df['CPI_inflation_rate'] = 2.9

# Select only the required columns in the correct order
final_input = merged_df[['open', 'high', 'low', 'close', 'volume', 'SMA', 'EMA',
                         'GDP', 'CPI_inflation_rate', 'positive', 'neutral', 'negative']]




# Load the trained model
model = load_model("Attention_Transformer_LSTM_Model.h5")

# Load the scalers
scaler_stock = joblib.load("scaler_stock.pkl")
scaler_sentiment = joblib.load("scaler_sentiment.pkl")
scaler_close = joblib.load("scaler_close.pkl")

# Select the last 10 days of relevant features
final_input = merged_df[['open', 'high', 'low', 'close', 'volume', 'SMA', 'EMA',
                         'GDP', 'CPI_inflation_rate', 'positive', 'neutral', 'negative']].tail(10)

# Scale stock & economic features
stock_economic_scaled = scaler_stock.transform(final_input.iloc[:, :-3])  
sentiment_scaled = scaler_sentiment.transform(final_input.iloc[:, -3:])

# Reshape for the model
X_stock_economic = np.array([stock_economic_scaled])  # Shape: (1, 10, 9)
X_sentiment = np.array([sentiment_scaled])  # Shape: (1, 10, 3)

# Predict the next day's closing price
predicted_scaled = model.predict([X_stock_economic, X_sentiment])

# Ensure the prediction is in correct shape
predicted_scaled = np.array(predicted_scaled).reshape(-1, 1)

# Inverse transform to get actual closing price
predicted_actual = scaler_close.inverse_transform(predicted_scaled)

print(f"Predicted Closing Price for Next Day: {predicted_actual[0][0]:.2f}")



# Get current and predicted close prices
current_price = float(final_input.iloc[-1, 3])  # Today's actual close
predicted_price = float(predicted_actual[0][0])  # Tomorrow's predicted close

# Calculate price change and percentage
price_change = predicted_price - current_price
change_percent = (price_change / current_price) * 100

# Thresholds (adjustable)
bearish_thresh = -0.5
bullish_thresh = 0.5

# Initialize probabilities using soft heuristics
if change_percent < bearish_thresh:
    bearish = min(100, abs(change_percent * 2))
    neutral = max(0, 100 - bearish)
    bullish = 0
elif change_percent > bullish_thresh:
    bullish = min(100, abs(change_percent * 2))
    neutral = max(0, 100 - bullish)
    bearish = 0
else:
    neutral = max(0, 100 - abs(change_percent * 2))
    bearish = bullish = abs(change_percent)

# Normalize to sum to 100%
total = bearish + neutral + bullish
bearish = round((bearish / total) * 100, 2)
neutral = round((neutral / total) * 100, 2)
bullish = round((bullish / total) * 100, 2)

# Print final trend output
print(f"\n📈 Prediction Summary")
print(f"Today's Closing Price: {current_price:.2f}")
print(f"Tomorrow's Predicted Closing Price: {predicted_price:.2f}")
print("Trend Probabilities:")
print(f"  Bearish (%): {bearish:.2f}%")
print(f"  Neutral (%): {neutral:.2f}%")
print(f"  Bullish (%): {bullish:.2f}%")



[*********************100%***********************]  1 of 1 completed


Price            Close        High         Low        Open     Volume  \
Ticker            AAPL        AAPL        AAPL        AAPL       AAPL   
Date                                                                    
2025-03-10  227.479996  236.160004  224.220001  235.539993   72071200   
2025-03-11  220.839996  225.839996  217.449997  223.809998   76137400   
2025-03-12  216.979996  221.750000  214.910004  220.139999   62547500   
2025-03-13  209.679993  216.839996  208.419998  215.949997   61368300   
2025-03-14  213.490005  213.949997  209.580002  211.250000   60107600   
2025-03-17  214.000000  215.220001  209.970001  213.309998   48073400   
2025-03-18  212.690002  215.149994  211.490005  214.160004   42432400   
2025-03-19  215.240005  218.759995  213.750000  214.220001   54385400   
2025-03-20  214.100006  217.490005  212.220001  213.990005   48862900   
2025-03-21  218.270004  218.839996  211.279999  211.559998   94127800   
2025-03-24  220.729996  221.479996  218.580002  221

✅ Sentiment data categorized successfully!
{
    "2025-04-29": {
        "positive": 0.0,
        "neutral": 0.2,
        "negative": 0.8
    },
    "2025-04-28": {
        "positive": 0.6,
        "neutral": 0.4,
        "negative": 0.0
    },
    "2025-04-27": {
        "positive": 0.74,
        "neutral": 0.26,
        "negative": 0.0
    },
    "2025-04-26": {
        "positive": 0.0,
        "neutral": 1.0,
        "negative": 0.0
    },
    "2025-04-25": {
        "positive": 0.0,
        "neutral": 1.0,
        "negative": 0.0
    },
    "2025-04-24": {
        "positive": 0.51,
        "neutral": 0.49,
        "negative": 0.0
    },
    "2025-04-23": {
        "positive": 0.52,
        "neutral": 0.48,
        "negative": 0.0
    },
    "2025-04-22": {
        "positive": 0.0,
        "neutral": 1.0,
        "negative": 0.0
    },
    "2025-04-21": {
        "positive": 0.0,
        "neutral": 1.0,
        "negative": 0.0
    },
    "2025-04-20": {
        "positive": 0.99,
   